In [2]:
import os
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [3]:
# Ana dataset klasörü
dataset_path = "./dataset/Videos/Videos/"

# Normal ve Abnormal klasörlerini al
normal_path = os.path.join(dataset_path, "normal")
abnormal_path = os.path.join(dataset_path, "abnormal")

# Tüm sınıfları belirleyelim
abnormal_classes = [folder for folder in os.listdir(abnormal_path) if os.path.isdir(os.path.join(abnormal_path, folder))]
class_to_index = {cls: idx + 1 for idx, cls in enumerate(abnormal_classes)}  # Abnormal sınıfları 1, 2, 3... olarak etiketliyoruz
class_to_index["normal"] = 0  # Normal aktiviteleri 0 olarak etiketliyoruz

# DataFrame oluşturmak için liste
data = []

# Normal videoları ekleyelim
for subfolder in os.listdir(normal_path):
    subfolder_path = os.path.join(normal_path, subfolder)
    if os.path.isdir(subfolder_path):
        videos = [os.path.join(subfolder_path, v) for v in os.listdir(subfolder_path) if v.endswith(('.mp4', '.avi', '.mov'))]
        data.extend([(video, "normal", 0) for video in videos])  # Label = 0

# Abnormal videoları ekleyelim
for subfolder in abnormal_classes:
    subfolder_path = os.path.join(abnormal_path, subfolder)
    videos = [os.path.join(subfolder_path, v) for v in os.listdir(subfolder_path) if v.endswith(('.mp4', '.avi', '.mov'))]
    data.extend([(video, subfolder, class_to_index[subfolder]) for video in videos])  # Label = 1, 2, 3, ...

# DataFrame oluştur
df = pd.DataFrame(data, columns=["video_path", "category", "label"])


In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"],random_state=42)

print(f"Train set {len(train_df)} videos")
print(f"Test set {len(test_df)} videos")

Train set 2224 videos
Test set 556 videos


# Buraya preprocessing adımı olarak Veri döndürme, zoomout, kontrast vs eklenebilir

In [6]:
import torch
import torchvision
import torch.nn.functional as F
from torch.utils.data import Dataset

class VideoDataset(Dataset):
    def __init__(self, df, num_frames=16, transform=None, resize_size=(192, 192)):
        self.df = df
        self.num_frames = num_frames
        self.transform = transform
        self.resize_size = resize_size  # Sabit video boyutu (H, W)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        video_path = self.df.iloc[idx]["video_path"]
        label = self.df.iloc[idx]["label"]

        # GPU hızlandırmalı video okuma
        frames = self.extract_frames(video_path, self.num_frames)

        # PyTorch formatına dönüştür (C, T, H, W)
        frames = frames.permute(3, 0, 1, 2)  # (T, H, W, C) → (C, T, H, W)

        # Eğer transform varsa uygula
        if self.transform:
            frames = self.transform(frames)

        return frames, torch.tensor(label, dtype=torch.long)

    def extract_frames(self, video_path, num_frames):
        # Torchvision VideoReader kullanarak GPU hızlandırmalı okuma
        vr = torchvision.io.VideoReader(video_path, "video")
        vr.set_current_stream("video")  # Video stream'ini seç

        frames = []
        for frame in vr:
            frame = frame['data']  # Frame'i al (C, H, W)
            frame = frame.float() / 255.0  # Normalize et (0-1)

            # **🔥 HATA ÇÖZÜMÜ: Tüm frame'leri aynı boyuta getir**
            frame = F.interpolate(frame.unsqueeze(0), size=self.resize_size, mode="bilinear", align_corners=False).squeeze(0)

            frames.append(frame.permute(1, 2, 0))  # (C, H, W) → (H, W, C)

            if len(frames) >= num_frames:
                break

        # Eğer video kısa ise tekrar eden frame'ler ekleyerek tamamla
        while len(frames) < num_frames:
            frames.append(frames[-1])  # Son frame'i tekrar et

        frames = torch.stack(frames)  # Frame'leri birleştir
        frames = frames[:num_frames]  # Sabit sayıda frame al (T, H, W, C)

        return frames.to("cuda")  # GPU'ya taşı

In [7]:
train_dataset = VideoDataset(train_df, num_frames=32)
test_dataset = VideoDataset(test_df, num_frames=32)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(f"Train loader {len(train_loader)} batches")
print(f"Test loader {len(test_loader)} batches")

Train loader 139 batches
Test loader 35 batches


In [11]:
for videos, labels in train_loader:
    print(f"Videos shape:{videos.shape}, Labels shape: {labels.shape}")
    break;

Videos shape:torch.Size([16, 3, 32, 192, 192]), Labels shape: torch.Size([16])


X3D modelimizin output layerini tüm classlarımızı kapsayacak şekilde arttırmamız lazım
Son katman ise ResNetBasicHead altında proj olarak tanımlanmış.

In [8]:
# X3D Modelini Yükleme
import torch

model_name= "x3d_xs"
model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)

num_classes = len(train_df["category"].unique())

Using cache found in C:\Users\Bugra/.cache\torch\hub\facebookresearch_pytorchvideo_main


In [17]:
#Kernel küçültme
#Bunun amacı modelin orijinalinin videoları aşırı küçültmesi
#Burası mantıksız oldu


#for layer in model.modules():
#    if isinstance(layer, torch.nn.Conv3d):
#        if layer.kernel_size[1] > 3:
#            layer.kernel_size = (layer.kernel_size[0], 3, 3)
#            layer.padding = (1, 1, 1)

In [9]:

#Son katmanı değiştirme
model.blocks[-1].proj = torch.nn.Linear(model.blocks[-1].proj.in_features, num_classes)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print(f"Model loaded on {device}")

Model loaded on cuda


print(model.blocks[-1].proj)
import torch

model = model.to("cpu")
dummy_input = torch.randn(1, 3, 16, 160, 160)

torch.onnx.export(model, dummy_input, "x3d_xs.onnx", opset_version=11
                  ,do_constant_folding=True, input_names=["input"], output_names=["output"])

In [19]:
import torch
print(torch.version.cuda)
print(torch.__version__)
import sys

print(sys.executable)  # Çalışan python yolunu verir

print(f"using device: {device}")


11.8
2.5.1
C:\Users\Bugra\anaconda3\python.exe
using device: cuda


In [10]:
print(model)

Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv2plus1d(
        (conv_t): Conv3d(3, 24, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), bias=False)
        (conv_xy): Conv3d(24, 24, kernel_size=(5, 1, 1), stride=(1, 1, 1), padding=(2, 0, 0), groups=24, bias=False)
      )
      (norm): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(24, 24, kernel_size=(1, 1, 1), stride=(1, 2, 2), bias=False)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(24, 54, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (norm_a): BatchNorm3d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(54, 54, kernel_size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 1, 1), groups=54, bias=False)
            (nor

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

# 🎯 Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay= 1e-3) #Weight decay overfitting azaltır weightleri stabilize eder

# ⚡ Yeni PyTorch Formatına Uygun AMP (Automatic Mixed Precision)
scaler = torch.amp.GradScaler(device="cuda")

# 🛑 Early Stopping Parametreleri
patience = 3  # Kaç epoch boyunca iyileşme olmazsa durdursun
min_delta = 0.01  # İyileşme için gereken minimum fark
best_loss = float("inf")  # En iyi validation loss başta sonsuz olarak ayarlanır
counter = 0  # İyileşme olmayan epoch sayısı
accuracy_threshold = 85.0  # Accuracy'nin geçmesi gereken eşik

for epoch in range(40):
    model.train()
    running_loss = 0.0

    for videos, labels in train_loader:
        videos, labels = videos.to("cuda"), labels.to("cuda")

        optimizer.zero_grad()
        with torch.amp.autocast("cuda"):
            outputs = model(videos)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)  # Train loss'u hesapla

    # 📌 VALIDATION AŞAMASI
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for videos, labels in test_loader:
            videos, labels = videos.to("cuda"), labels.to("cuda")
            outputs = model(videos)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Accuracy hesapla
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(test_loader)  # Validation loss'u hesapla
    val_accuracy = 100 * correct / total  # Accuracy hesapla

    print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%")

    # 🛑 Early Stopping Kontrolü (Sadece accuracy > %85 ise)
    if val_accuracy >= accuracy_threshold:
        if best_loss - val_loss > min_delta:
            best_loss = val_loss
            counter = 0  # Eğer loss iyileştikçe sıfırla
            torch.save(model.state_dict(), "best_model.pth")  # En iyi modeli kaydet
        else:
            counter += 1  # İyileşme olmazsa artır
            print(f"❗ Early stopping counter: {counter}/{patience}")

        if counter >= patience:
            print("⏹️ Early stopping triggered. Training stopped.")
            break  # Eğitimden çık
    else:
        print(f"🚨 Accuracy {val_accuracy:.2f}% < {accuracy_threshold}%, early stopping devre dışı!")

    if((epoch + 1) % 5 == 0):
        torch.save({'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
                    }, f"checkpoint_{epoch}.pth")
        print(f"Model {epoch + 1} saved to checkpoint_{epoch + 1}.pth")

print("✅ Eğitim tamamlandı!")

Epoch 1 - Train Loss: 2.0968 | Val Loss: 2.0709 | Val Accuracy: 34.53%
🚨 Accuracy 34.53% < 85.0%, early stopping devre dışı!
Model 0 saved to checkpoint_0.pth
Epoch 2 - Train Loss: 1.8415 | Val Loss: 2.0268 | Val Accuracy: 32.37%
🚨 Accuracy 32.37% < 85.0%, early stopping devre dışı!
Epoch 3 - Train Loss: 1.6654 | Val Loss: 2.0890 | Val Accuracy: 33.09%
🚨 Accuracy 33.09% < 85.0%, early stopping devre dışı!
Epoch 4 - Train Loss: 1.5589 | Val Loss: 2.3445 | Val Accuracy: 33.45%
🚨 Accuracy 33.45% < 85.0%, early stopping devre dışı!
Epoch 5 - Train Loss: 1.4247 | Val Loss: 2.1574 | Val Accuracy: 32.19%
🚨 Accuracy 32.19% < 85.0%, early stopping devre dışı!
Epoch 6 - Train Loss: 1.2936 | Val Loss: 1.9907 | Val Accuracy: 36.15%
🚨 Accuracy 36.15% < 85.0%, early stopping devre dışı!
Model 5 saved to checkpoint_5.pth


KeyboardInterrupt: 